# Resultados gerais

In [1]:
import sqlite3
import pandas as pd
import numpy as np

dbconn = sqlite3.connect("Resultados.sqlite")

In [2]:
from generate_tsp import GerarProblemaRadialTSP
from sga import SGAPermutacao, SGAInteiroLimitado

In [3]:
def EscreverResultado(res,params,topologia,tipo):
    params['Topologia'] = topologia
    pop,stats,hof,params,custo = res
    resPerm = pd.Series(params)
    resPerm['Tipo'] = tipo
    minVal, gap, success = stats.select("min",'gap','success')
    resPerm['Sucesso'] = success[-1]
    resPerm['Geração final'] = len(success)
    resPerm['MBF'] = np.mean(minVal)
    resPerm['Gap'] = gap[-1]
    resPerm['Valor encontrado'] = hof[0].fitness.values[0]
    return pd.DataFrame([resPerm])
    
def Experimento(posicoes,params,nomePosicao='Padrão'):
    res = SGAPermutacao(posicoes,params)
    resPerm = EscreverResultado(res,params,nomePosicao,'Permutação')
    res = SGAInteiroLimitado(posicoes,params)
    resInt = EscreverResultado(res,params,nomePosicao,'Inteiro')
    return pd.concat([resPerm,resInt],ignore_index=True)

In [4]:
def CalcularSR(resultados):
    return (100*resultados.groupby(['Topologia','Tipo','P','mu'])['Sucesso'].sum()/resultados.groupby(['Topologia','Tipo','P','mu'])['Sucesso'].count()).unstack(['Topologia',"Tipo"])
CalcularMBF = lambda resultados: resultados.groupby(['Topologia','Tipo','P','mu'])['MBF'].describe().reset_index().set_index(['Topologia','Tipo','P','mu']).unstack(['Topologia','Tipo'])
CalcularAES = lambda resultados: resultados[resultados['Sucesso'] == True].groupby(['Topologia','Tipo','P','mu'])['Geração final'].describe().unstack(['Topologia','Tipo'])

In [5]:
from itertools import product
resultados = pd.DataFrame()
cidadesLista = [7,8,9]
individuosLista = [50]
geracoes = 300

for cidades, individuos in product(cidadesLista,individuosLista):
    posicoes = GerarProblemaRadialTSP(cidades)
    posicoesNormal = posicoes[:,:-1].copy()
    posicoesAleatorio = posicoesNormal.copy().T
    np.random.shuffle(posicoesAleatorio.T)
    posicoesAleatorio = posicoesAleatorio.T
    params = {
        "N":geracoes,
        "P":posicoes.shape[1],
        "mu":individuos,
        "cxpb": 0.3,
        "mupb":0.3,
        "muidxpb":0.3
    }
    for i in range(2):
        print(i)
        resultados = pd.concat([resultados,Experimento(posicoes,params,'Circular')],ignore_index=True)
        resultados = pd.concat([resultados,Experimento(posicoes[:,::-1],params,'Aleatório')],ignore_index=True)

0
1
0
1
0
1


In [6]:
CalcularSR(resultados)

Topologia Aleatório            Circular           
Tipo        Inteiro Permutação  Inteiro Permutação
P  mu                                             
8  50         100.0      100.0    100.0      100.0
9  50         100.0      100.0    100.0        0.0
10 50         100.0        0.0    100.0        0.0

In [21]:
from sga import CriarAlgoritmoSGAPermutacao,GerarFuncaoCustoPermutacao

toolbox = CriarAlgoritmoSGAPermutacao(5,0.1,GerarFuncaoCustoPermutacao(GerarProblemaRadialTSP(5)))

In [22]:
pop = toolbox.population(5)

In [25]:
toolbox.evaluate(pop[2])

(8.057480106940814,)

In [52]:
from itertools import product

cidadesLista = [10,11,12,13]
individuosLista = [40,50,60,70]
geracoes = 300

for cidades, individuos in product(cidadesLista,individuosLista):
    posicoes = GerarProblemaRadialTSP(cidades)
    posicoes = posicoes[:,:-1]
    params = {
        "N":geracoes,
        "P":posicoes.shape[1],
        "mu":individuos,
        "cxpb": 0.3,
        "mupb":0.3,
        "muidxpb":0.3
    }
    for i in range(500):
        resultados = pd.concat([resultados,Experimento(posicoes,params,'Circular')],ignore_index=True)
        resultados = pd.concat([resultados,Experimento(posicoes[:,::-1],params,'Reverso')],ignore_index=True)

IndentationError: unexpected indent (586792398.py, line 20)

In [ ]:
from itertools import product

cidadesLista = [10,11,12]
individuosLista = [100]
geracoes = 300

for cidades, individuos in product(cidadesLista,individuosLista):
    posicoes = GerarProblemaRadialTSP(cidades)
    posicoes = posicoes[:,:-1]
    params = {
        "N":geracoes,
        "P":posicoes.shape[1],
        "mu":individuos,
        "cxpb": 0.3,
        "mupb":0.3,
        "muidxpb":0.3
    }
    
    for i in range(500):
        resultados = pd.concat([resultados,Experimento(posicoes,params,'Circular')],ignore_index=True)
        resultados = pd.concat([resultados,Experimento(posicoes[:,::-1],params,'Reverso')],ignore_index=True)

In [43]:
resultados2 = resultados.iloc[15000:].copy()

In [46]:
resultados2

,N,P,mu,cxpb,mupb,muidxpb,Topologia,Tipo,Sucesso,Geração final,MBF,Gap,Valor encontrado
15000,300,5,10,0.3,0.3,0.3,Reverso,Permutação,True,2,5.877853,1.000000,5.877853
15001,300,5,10,0.3,0.3,0.3,Reverso,Inteiro,True,5,7.040321,1.000000,5.877853
15002,300,5,10,0.3,0.3,0.3,Reverso,Permutação,True,3,6.846576,1.000000,5.877853
15003,300,5,10,0.3,0.3,0.3,Reverso,Inteiro,True,2,5.877853,1.000000,5.877853
15004,300,5,10,0.3,0.3,0.3,Reverso,Permutação,True,2,6.604395,1.000000,5.877853
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,300,9,30,0.3,0.3,0.3,Reverso,Inteiro,True,22,8.313924,1.000000,6.156363
29996,300,9,30,0.3,0.3,0.3,Reverso,Permutação,False,301,9.732813,0.651091,7.359432
29997,300,9,30,0.3,0.3,0.3,Reverso,Inteiro,True,5,8.504459,1.000000,6.156363
29998,300,9,30,0.3,0.3,0.3,Reverso,Permutação,False,301,9.756370,0.659913,7.359432


In [3]:
dbconn = sqlite3.connect("resultados_finais.sqlite")
resultados = pd.read_sql("SELECT * FROM resultados",dbconn)

## TODO 

* Mudar a função de mutação para espelhamento
* Verificar se a função de roleta está funcionando (mudar para torneio)
* Para novo mapa de soluções, embaralhar os índices aleatoriamente